#### This code will take in list of drugs aimed at targetting Prostate Cancer disease and output the rank ordered list of them based on LINCS(http://www.ilincs.org/ilincs/) gene signature reversal. It does so by automatically extracting drug gene signatures from the iLINCS website and computing the discordance wrt to the disease gene signature<br>

#### Inputs: 
#### 1. Drug list 
#### 2. Differential Gene expression of Prostate Cancer disease
        
#### Outputs: 
#### 1. Rank ordered list of drugs based on gene signature reversal

#### ----------------------------------------------------------------------------------------------------------------------

The following function takes hold of latest downloaded file and renames it to preferred extension that is easy to process

In [1]:
import os
import time
def latest_download_file():
      path = '/Users/keshavpatil/Downloads'  #change this to wherever your files get downloaded by default
      os.chdir(path)
      files = sorted(os.listdir(os.getcwd()), key=os.path.getmtime)
      newest = files[-1]

      return newest

fileends = "crdownload"
while "crdownload" == fileends:
    time.sleep(1)
    newest_file = latest_download_file()
    if "crdownload" in newest_file:
        fileends = "crdownload"
    else:
        fileends = "none"

The following function checks if certain button is available on the webpage

In [2]:
def existsElement(driver, s):

    try:   
        button = driver.find_element_by_partial_link_text('Found 0 of LINCS chemical perturbagen signatures')     
    except NoSuchElementException:
        return True
    return False

- The following cell uses python package called selenium (you have to install this!) which is an excellent tool for webscrapping. The code
used Chrome Driver (you have to install this!) to automate the data extraction from the web page <br>
- This cell looks for the drug name and reports back if the drug name is not found and moves to the next drug in the list, also if the drug name is found but the certain unique conditions of the experiments are not available it reports back and moves to the next drug in the list <br>
- The webdriver looks up for specific buttons to be clicked by unique identifiers found in the source code (HTML, CSS) of the webpage

In [3]:
from selenium import webdriver
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


# pickup the drug names as input
df_name = pd.read_excel('/Users/keshavpatil/Desktop/RebootRx/PRAD/Drug_names.xlsx') #change this to path where your drug names file is stored
d  = np.array(df_name)

sign_id = []
d_name = []


# loop through each drug to extract signature ID of ech drug
for i in range(0,len(df_name)): 
    driver = webdriver.Chrome(ChromeDriverManager().install()) 
    #driver = webdriver.Chrome("/Users/keshavpatil/Downloads/chromedriver") 
    driver.get("http://www.ilincs.org/ilincs/signatures/search/"+ d[i][0].lower()) 
    time.sleep(10)
    q = existsElement(driver, d[i][0].lower())
    if q:
        button = driver.find_element_by_partial_link_text('of LINCS chemical perturbagen signatures') 
        button.click()
        l = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//button[@data-title='Signature selection']")))
        l.click()
        l1 = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//button[@title='Select all found signatures']")))
        l1.click()
        l2 = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//i[@class='fa fa-download']")))
        l2.click()
        l3 = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//button[@title='Download all found signatures results table.']")))
        l3.click()
        
        time.sleep(10)
        file = latest_download_file()
        my_file = file
        base = os.path.splitext(my_file)[0]
        os.rename(my_file, base + '.csv')
        
        
        df_drug = pd.read_csv('/Users/keshavpatil/Downloads/'+ base + '.csv',sep='\t')
        
        print(i)
              
    else:
        print("Drug name {a} not found in the LINCS... moving to the next drug in the list".format(a={d[i][0].lower()}))
        driver.close() 
        continue
         
    driver.close() 
    
    
    if ((df_drug['CellLine'] == 'PC3') & (df_drug['Time'] == '24h') & (df_drug['Concentration'] == '10uM') & (df_drug['Perturbagen'] == d[i][0])).any():     
    # setting the experimental conditions to extract the needed signature ID        
        df_extract1 = df_drug[df_drug['CellLine'] == 'PC3']
        df_extract2 = df_extract1[df_extract1['Time'] == '24h']
        df_extract3 = df_extract2[df_extract2['Concentration'] == '10uM']
        df_extract4 = df_extract3[df_extract3['Perturbagen'] == d[i][0]]
        d_name.append(d[i][0].lower())

    else:
        print("Set conditions for the drug name not found in the LINCS... moving to the next drug in the list")
        continue


    a = np.array(df_extract4)
    sign_id.append(a[0][0])
    print(sign_id)





====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======


0
['LINCSCP_161324']


Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


1
Set conditions for the drug name not found in the LINCS... moving to the next drug in the list


Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


...

11
Set conditions for the drug name not found in the LINCS... moving to the next drug in the list


Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606


12
['LINCSCP_161324', 'LINCSCP_152893', 'LINCSCP_152947', 'LINCSCP_44017', 'LINCSCP_272018']


Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


13
Set conditions for the drug name not found in the LINCS... moving to the next drug in the list


The following cell contains the function to compute the discordance of the drugs whose data has been successfully extracted
https://www.nature.com/articles/s41467-018-07659-z.pdf

In [5]:
# Compute discordance for the drugs with respect to disease
def discordance(df_gene_drug):
    df_disease = pd.read_csv('/Users/keshavpatil/Desktop/RebootRx/PRAD/PRAD_MatchedSample_DESeqResults.csv')
    
    numerator = 1
    denominator = 1
    for j in range(0,len(df_gene_drug)):
        for k in range(0,len(df_disease)):
            if df_gene_drug['Name_GeneSymbol'].values[j] == df_disease['Unnamed: 0'].values[k].split('|')[0]:
                if np.sign(df_gene_drug['Value_LogDiffExp'].values[j]) == np.sign(df_disease['log2FoldChange'].values[k]):
                    denominator = denominator + 1
                    numerator = numerator + 0
                else:
                    numerator = numerator + 1
                    denominator = denominator + 0
    discordance = numerator/denominator
    
    return discordance
                                 

The following cell downloads the gene signature of the drugs based on the signature IDs extracted earlier

In [6]:

# download df_gene_drug
discord = []
for i in range(0,len(sign_id)):
    
    
    
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get("http://www.ilincs.org/ilincs/signature/"+sign_id[i]) 

    time.sleep(10) 
    
    l6 = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//i[@class='fa fa-download']")))
    l6.click()
    l7 = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, "complete signature (xls)")))
    l7.click()
    
    time.sleep(10)
    file = latest_download_file()
    my_file = file
    base = os.path.splitext(my_file)[0]
    os.rename(my_file, base + '.csv')

    df_gene_drug = pd.read_csv('/Users/keshavpatil/Downloads/'+ base + '.csv',sep='\t')  
    
    # compute discordance and save it
    discord.append(discordance(df_gene_drug))
    print(discord)
    
    driver.close()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606


[1.055793991416309]


Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======


[1.055793991416309, 1.259433962264151]


Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


====== WebDriver manager ======


[1.055793991416309, 1.259433962264151, 1.2647754137115839]


Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


[1.055793991416309, 1.259433962264151, 1.2647754137115839, 1.0382978723404255]




====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
Driver [/Users/keshavpatil/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


[1.055793991416309, 1.259433962264151, 1.2647754137115839, 1.0382978723404255, 1.0083857442348008]


The following cell finally returns the rank ordered list of the drugs

In [7]:
# Display order
dataset = pd.DataFrame({'drug_name': d_name, 'discordance': discord}, columns=['drug_name', 'discordance'])
drug_sorted = dataset.sort_values(["discordance", "drug_name"], ascending=False)
drug_sorted

,drug_name,discordance
2,dutasteride,1.264775
1,calcitriol,1.259434
0,metformin,1.055794
3,digoxin,1.038298
4,ganciclovir,1.008386


#### ----------------------------------------------------------- THE END ----------------------------------------------------------